In [ ]:
import numpy as np
import tensorflow as tf
import samples as ls
import math
import model_types

#new models to test: 10105, (20)*5

step_counter = 0
load_rnn = False
load_samples = True
model_num = 10105
overlap_step = 1

if model_num == 0: #SIMPLE TRUNC, adam
    #number of dimensions used and steps when unrolling
    input_dim = 6
    input_steps = None
    unroll_steps = 50
    use_lstm = False
    #if false, only label of the last output is compared
    #else label at each timestep is compared
    use_labels_data = False
    use_ctc = False
    hidden_layers = [50]
    labels,sample_ids,filename = model_types.get_labels(0)
elif model_num == 100: #SIMPLE TRUNC, adam
    input_dim = 6
    input_steps = None
    unroll_steps = 30
    overlap_step = 15
    use_lstm = True
    use_labels_data = False
    use_ctc = False
    hidden_layers = [20]
    labels,sample_ids,filename = model_types.get_labels(0)
elif model_num == 10100: #SIMPLE TRUNC, adam
    input_dim = 6
    input_steps = None
    unroll_steps = 20
    use_lstm = False
    use_labels_data = False
    use_ctc = False
    hidden_layers = [50]
    labels,sample_ids,filename = model_types.get_labels(0)
elif model_num == 7: #SIMPLE TRUNC, adam
    input_dim = 6
    input_steps = None
    unroll_steps = 30
    use_lstm = False
    use_labels_data = False
    use_ctc = False
    hidden_layers = [50]
    labels,sample_ids,filename = model_types.get_labels(2)
elif model_num == 107: #SIMPLE TRUNC, adam
    input_dim = 6
    input_steps = None
    unroll_steps = 20
    use_lstm = False
    use_labels_data = False
    use_ctc = False
    hidden_layers = [50]
    labels,sample_ids,filename = model_types.get_labels(2)
elif model_num == 10107: #SIMPLE TRUNC, adam
    input_dim = 6
    input_steps = 50
    unroll_steps = 50
    overlap_step = 25
    use_lstm = False
    use_labels_data = True
    use_ctc = False
    hidden_layers = [50]
    labels,sample_ids,filename = model_types.get_labels(2)
elif model_num == 1010107: #SIMPLE TRUNC, adam
    input_dim = 6
    input_steps = 30
    unroll_steps = 30
    overlap_step = 15
    use_lstm = True
    use_labels_data = True
    use_ctc = False
    hidden_layers = [50]
    labels,sample_ids,filename = model_types.get_labels(2)
elif model_num == 5: #SIMPLE TRUNC, adam
    input_dim = 6
    input_steps = None
    unroll_steps = 50
    use_lstm = False
    use_labels_data = False
    use_ctc = False
    hidden_layers = [50]
    labels,sample_ids,filename = model_types.get_labels(1)
elif model_num == 105: #SIMPLE TRUNC, adam
    input_dim = 6
    input_steps = None
    unroll_steps = 50
    use_lstm = True
    use_labels_data = False
    use_ctc = False
    hidden_layers = [50]
    labels,sample_ids,filename = model_types.get_labels(1)
elif model_num == 10105: #SIMPLE TRUNC, adam
    input_dim = 6
    input_steps = None
    unroll_steps = 100
    overlap_step = 50
    use_lstm = False
    use_labels_data = False
    use_ctc = False
    hidden_layers = [50]
    labels,sample_ids,filename = model_types.get_labels(1)
elif model_num == 1010105: #SIMPLE TRUNC, adam
    input_dim = 6
    input_steps = None
    unroll_steps = 10
    use_lstm = False
    use_labels_data = False
    use_ctc = False
    hidden_layers = [50]
    labels,sample_ids,filename = model_types.get_labels(1)
elif model_num == 101010105: #SIMPLE TRUNC, adam
    input_dim = 6
    input_steps = None
    unroll_steps = 20
    overlap_step = 10
    use_lstm = False
    use_labels_data = False
    use_ctc = False
    hidden_layers = [20]
    labels,sample_ids,filename = model_types.get_labels(1)
elif model_num == 205: #SIMPLE TRUNC, adam
    input_dim = 6
    input_steps = 20
    unroll_steps = 20
    use_lstm = False
    use_labels_data = True
    use_ctc = False
    hidden_layers = [50]
    labels,sample_ids,filename = model_types.get_labels(1)
elif model_num == 20205: #SIMPLE TRUNC, adam
    input_dim = 6
    input_steps = 50
    unroll_steps = 50
    overlap_step = 25
    use_lstm = False
    use_labels_data = True
    use_ctc = False
    hidden_layers = [50]
    labels,sample_ids,filename = model_types.get_labels(1)
elif model_num == 2020205: #SIMPLE TRUNC, adam
    input_dim = 6
    input_steps = 100
    unroll_steps = 100
    overlap_step = 50
    use_lstm = True
    use_labels_data = True
    use_ctc = False
    hidden_layers = [50]
    labels,sample_ids,filename = model_types.get_labels(1)
elif model_num == 10:#unused
    input_dim = 6
    input_steps = None
    unroll_steps = None
    use_lstm = False
    use_labels_data = False
    use_ctc = True
    hidden_layers = [50]
    labels,sample_ids,filename = model_types.get_labels(3)
else:
    raise ValueError('Unknown model')
    
classes = len(labels)
if use_ctc: classes += 1

#percentage used for test data
test_rate = 0.1

learning_rate = 0.001

#some assertion which might catch wrong config (not guaranteed)
if use_labels_data:
    assert input_steps is not None
if use_ctc:
    assert input_steps is None
    assert not use_labels_data
    assert 'straight' not in labels

In [ ]:
if load_samples:
    samples = ls.load(filename)
else:
    samples = ls.Samples(labels)
    
    for label in labels:
        samples.load_samples(label = label, sample_ids = sample_ids)

    samples.convert_to_input(size = None, accel='xyz', gyro='xyz', compass='')

samples.unroll(unroll_steps, overlap_step)
if not use_ctc: samples.convert_to_onehot()
samples.split_test(test_rate)

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, input_steps, input_dim], name='x')
seqlen = tf.placeholder(tf.int32, [None])

Ws = []
for i in range(len(hidden_layers)-1):
    W = tf.Variable(tf.random_normal([hidden_layers[i], hidden_layers[i+1]]))
    b = tf.Variable(tf.zeros(hidden_layers[i+1]))
    Ws.append((W,b))
W = tf.Variable(tf.random_normal([hidden_layers[-1], classes]))
b = tf.Variable(tf.zeros([classes]))
Ws.append((W,b))

if use_lstm:
    initial_state = tf.placeholder(tf.float32, [None, hidden_layers[0]*2], name='state')
    zero_state = np.zeros([hidden_layers[0]*2], np.float32)
    rnn_cell = tf.contrib.rnn.BasicLSTMCell(hidden_layers[0], 
                                            forget_bias=0.0, state_is_tuple=False)
else:
    initial_state = tf.placeholder(tf.float32, [None, hidden_layers[0]], name='state')
    zero_state = np.zeros([hidden_layers[0]], np.float32)
    rnn_cell = tf.contrib.rnn.BasicRNNCell(hidden_layers[0])
    
#TODO dropout and multirnn
if input_steps is None:
    outputs, state = tf.nn.dynamic_rnn(rnn_cell, x, sequence_length=seqlen,
                                   initial_state=initial_state, dtype=tf.float32)
else:
    outputs, state = tf.nn.dynamic_rnn(rnn_cell, x,
                                   initial_state=initial_state, dtype=tf.float32)

if use_labels_data:
    #before: [batchsize, timesteps, classes]
    #then: list of size batchsize of [timesteps, classes] 
    #after concat: [batchsize*timesteps, classes]
    y = tf.placeholder(tf.float32, [None, input_steps, classes], name='y')
    ys = tf.unstack(y, axis=1)
    y2 = tf.concat(ys, axis=0)
    outputs2 = tf.unstack(outputs, axis=1)
    
    #calc outputs
    rnn_outs = []
    for out in outputs2:
        preactivations = []
        activations = []
        act = out
        for W,b in Ws:
            preact = tf.matmul(act, W) + b
            preactivations.append(preact)
            act = tf.nn.relu(preact)
            #act = tf.sigmoid(preact)
            activations.append(act)
        rnn_outs.append(preact)
    #prediction for the last timestep
    rnn_final = tf.nn.softmax(preact)
    activations[-1] = rnn_final
    rnn_pred = tf.argmax(rnn_final, 1, name='RNN_pred')
    
    #predictions for all timestep (needed for loss and accuracy)
    preact = tf.concat(rnn_outs, axis=0)
    rnn = tf.nn.softmax(preact)
elif use_ctc:
    y = tf.sparse_placeholder(tf.int32, name='y')
    outputs = tf.reshape(outputs, [-1, hidden_layers[0]])

    preactivations = []
    activations = []
    act = outputs
    for W,b in Ws:
        preact = tf.matmul(act, W) + b
        preactivations.append(preact)
        act = tf.nn.relu(preact)
        #act = tf.sigmoid(preact)
        activations.append(act)
    rnn = tf.nn.softmax(preact)
    activations[-1] = rnn
    rnn_pred = tf.argmax(rnn, 1, name='RNN_pred')
    preact = tf.reshape(preact, [tf.shape(x)[0],-1,classes])
    
else:
    y = tf.placeholder(tf.float32, [None, classes], name='y')
    y2 = y
    outputs = tf.transpose(outputs, [1,0,2])

    preactivations = []
    activations = []
    act = outputs[-1]
    for W,b in Ws:
        preact = tf.matmul(act, W) + b
        preactivations.append(preact)
        act = tf.nn.relu(preact)
        #act = tf.sigmoid(preact)
        activations.append(act)
    rnn = tf.nn.softmax(preact)
    activations[-1] = rnn
    rnn_pred = tf.argmax(rnn, 1, name='RNN_pred')

#important: logits=op before calling softmax!!
if use_ctc:
    ctc_loss = tf.nn.ctc_loss(inputs=preact, labels=y, sequence_length=seqlen, time_major=False)
    loss = tf.reduce_mean(ctc_loss)
    decoded, log_prob = tf.nn.ctc_greedy_decoder(preact, seqlen)
    accuracy = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32),
                                          y))
else:
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y2, logits=preact))
    correct_prediction = tf.equal(tf.argmax(y2,1), tf.argmax(rnn,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


In [ ]:
sess = tf.Session()
saver = tf.train.Saver()

#summaries
def var_summary(var, name):
    mean = tf.reduce_mean(var)
    tf.summary.scalar(name + '_mean', mean)
    stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar(name + '_stddev', stddev)
    tf.summary.scalar(name + '_max', tf.reduce_max(var))
    tf.summary.scalar(name + '_min', tf.reduce_min(var))
    tf.summary.histogram(name + '_histogram', var)
i = 0
for W,b in Ws: 
    var_summary(W, 'weights_%s' % i)
    var_summary(b, 'bias_%s' % i)
    i += 1
for i in range(len(activations)):
    tf.summary.histogram('preactivations_%s' % i, preactivations[i])
    tf.summary.histogram('activations_%s' % i, activations[i])
tf.summary.scalar('ctc_loss' if use_ctc else 'cross_entropy', loss)
tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('tmp/rnn/summary-%s' % model_num, sess.graph)

#init or load from checkpoint
init = tf.global_variables_initializer()
sess.run(init)

#save model meta graph
meta_graph_def = tf.train.export_meta_graph(filename='tmp/rnn/model/rnn-%s.meta' % model_num)

if load_rnn:
    saver.restore(sess, 'tmp/rnn/model/rnn-%s/rnn-%s-%s' % (model_num,model_num,step_counter))

    

In [ ]:
#not needed anymore
def convert_to_onehot(labels):
    new_labels = []
    for y in labels:
        label = [0]*classes
        label[y] = 1
        new_labels.append(label)
    return new_labels

def sparse_tuple_from(sequences, dtype=np.int32):
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n]*len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1]+1], dtype=np.int64)

    return indices, values, shape

def test_all():
    xtest,ytest,seq = samples.get_all(use_labels_data=use_labels_data)
    #ytest = convert_to_onehot(ytest)
    if use_ctc: 
        ytest2 = []
        for yk in ytest:
            ytest2.append([yk])
        ytest = sparse_tuple_from(ytest2)
    acc = sess.run(accuracy, feed_dict={x: xtest, y: ytest, seqlen: seq,
                                        initial_state: [zero_state]*len(xtest)})
    print("Accuracy: %s" % acc)
    
def test():
    xtest,ytest,seq = samples.get_test(use_labels_data=use_labels_data)
    #ytest = convert_to_onehot(ytest)
    acc = sess.run(accuracy, feed_dict={x: xtest, y: ytest, seqlen: seq,
                                        initial_state: [zero_state]*len(xtest)})
    print("Accuracy: %s" % acc)
    
def per_label_acc(test=True):
    if test:
        xtest,ytest,seq = samples.get_test(use_labels_data=use_labels_data)
    else:
        xtest,ytest,seq = samples.get_all(use_labels_data=use_labels_data)
    accs = [0] * classes
    counts = [0] * classes

    for i in range(len(ytest)):
        acc = sess.run(accuracy, feed_dict={x: [xtest[i]], y: [ytest[i]], seqlen: [seq[i]],
                                        initial_state: [zero_state]})
        c = 0
        if use_labels_data:
            for y_oh in ytest[i]:
                for k in range(len(y_oh)):
                    if y_oh[k] > 0.5:
                        c = k
                        break
                accs[c] += acc
                counts[c] += 1
        else:
            for k in range(len(ytest[i])):
                if ytest[i][k] > 0.5:
                    c = k
                    break
            accs[c] += acc
            counts[c] += 1

    for i in range(classes):
        acc = accs[i]/counts[i]
        print(counts[i])
        print('%s: %s' % (labels[i],acc))

def train(steps, batch_size):
    global step_counter
    progress = 5
    for i in range(steps):
        xtrain,ytrain,seq = samples.get_batch(batch_size,
                                              use_labels_data=use_labels_data)
        #ytrain = convert_to_onehot(ytrain)
        feed = {x: xtrain, y: ytrain, 
                seqlen: seq, initial_state: [zero_state]*len(xtrain)}
        summary,_ = sess.run([merged, optimizer], feed_dict=feed)
        train_writer.add_summary(summary, i+step_counter)
        if i>=progress/100.0*steps-1:
            print("Progress: %s%%" % progress)
            progress += 5
            test()
            saver.save(sess, 'tmp/rnn/model/rnn-%s/rnn-%s' % (model_num,model_num), global_step=step_counter)
    step_counter += steps
    saver.save(sess, 'tmp/rnn/model/rnn-%s/rnn-%s' % (model_num,model_num), global_step=step_counter)

In [ ]:
if input_steps is None:
    def feed_data(data):
        return sess.run(rnn_pred, feed_dict={x:[data], seqlen:[len(data)], initial_state:[zero_state]})

    old_state = zero_state

    def reset():
        global old_state
        old_state = zero_state

    def feed_seq(s):
        global old_state
        res,new_state = sess.run([rnn_pred,state], feed_dict={x:[s], seqlen:[len(s)], initial_state:[old_state]})
        old_state = new_state[-1]
        return res[-1]
else:
    def feed_data(data):
        return sess.run(rnn_pred, feed_dict={x:[data], seqlen:[len(data)], initial_state:[zero_state]})
    
    old_state = zero_state
    xs = []
    
    def reset():
        global old_state
        old_state = zero_state
    
    def feed_seq(s):
        global old_state
        global xs
        xs.extend(s)
        if len(xs) > input_steps:
            del xs[:len(xs)-input_steps]
            assert len(xs) == input_steps
            
        if len(xs) == input_steps:
            res,new_state = sess.run([rnn_pred,state], feed_dict={x:[xs], 
                                seqlen:[len(xs)], initial_state:[old_state]})
            old_state = new_state[-1]
            return res
        else:
            return 0
        
    def feed_one(s):
        global old_state_second
        global old_state_last
        res,new_state = sess.run([rnn_pred,state], feed_dict={x:[s], 
                            seqlen:[len(s)], initial_state:[old_state]})
        old_state = new_state[-1]
        return res

In [ ]:
#training
if True:
    test_all()
    train(1000,100)
    test_all()

In [ ]:
if True:
    per_label_acc()

In [ ]:
#tests whether the network only outputs zero (to make sure accuracy isnt skewed)
if False:
    index = 3
    for i in range(len(labels)):
        print('%s: %s' % (i,labels[i]))
    print('should be %s - %s' % (index, labels[index]))
    sample = samples.data_old[labels[index]][40]
    count = 0
    for i in range(10,len(sample)):
        res = feed_data(sample[:i])
        if res == index: count += 1
    print(count)

In [ ]:
#proof of memory
if False:
    a = [0,0,0,0,0,0]
    b = [0,2,0,5,-100,0]
    
    reset()
    data1 = [b,a,a,a,a]
    print(feed_data(data1))
    reset()
    for i in data1:
        print(feed_seq([i]))
    print(feed_seq([a]))
    reset()
    for i in data1:
        print(feed_seq([i]))
    print(feed_seq([b]))

In [ ]:
#live demo
%env ROS_HOSTNAME=zinunb
%env ROS_IP=$(hostname -I)
%env ROS_MASTER_URI=http://pi:11311

if False:
    import imu_listener as il
    reset()
    il.imu_listener(feed_seq,labels,1)
    print('Exited')